In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


2023-10-29 11:04:37.119526: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 11:04:37.150343: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 11:04:37.150380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 11:04:37.150410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 11:04:37.156806: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 11:04:37.157663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# Load the dataset
data = pd.read_csv("../data/data.csv") 

In [3]:
# Convert the 'Date' column to a datetime object
data['Date'] = pd.to_datetime(data['Date'])

In [4]:
# Encode categorical variables (e.g., GemstoneName, Color, Clarity, Cut)
label_encoders = {}
for column in ['GemstoneName', 'Color', 'Clarity', 'Cut']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


In [5]:
# Split the data into features and target
X = data.drop(columns=['Price'])
y = data['Price']

In [ ]:
# Normalize numerical features
scaler = StandardScaler()
X[['Weight']] = scaler.fit_transform(X[['Weight']])